In [16]:
import OffsetArrays
using Plots
using Revise
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using IterTools
using DelimitedFiles
include("./src/bp.jl") 
include("./src/single_instance.jl") 
include("./src/post_infer.jl") 
include("./src/observables.jl") 
include("./src/utils.jl")

gam_vs_lam_optimal (generic function with 1 method)

# Population dynamics for the messages

In [17]:
# S=0   I=1
# The times go from ti = 0 to ti=T+1
# when ti=0 it means that i is the zero patient
# when ti=T+1 it means that i is never infected

T = 8 # discrete time
N = 30000; #population size
fr = 0.0 # noise of measure
grid = 21 # the grid of the phase diagram
γ = 0.2
λRange = LinRange(0.01,0.4,grid) 
dilRange = LinRange(0.0,1.0,grid) 

21-element LinRange{Float64, Int64}:
 0.0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,…,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0

In [ ]:
#here we choose what regime we want to study (λ VS dilution, λ VS γ, dilution VS fr ...)
edges = readdlm("./ia-escorts-dynamic.edges",Int,skipstart=2)[:,1:2];
G = build_real_graph(edges)
degree_dist = empirical_deg_distr(G) 
inf_out = inf_vs_dil_optimal(γ, λRange, N, T, degree_dist, fr , dilRange , tot_iterations = 40, count_obs = true )

In [ ]:
# Here we write the results in files.
samp = 1
graph = "real"

In [ ]:
open("./dil_lam_$(graph)_$(samp)/ninf.txt","w") do io
    writedlm(io,inf_out[:,:,1]) 
end
for t = 1:T+1
    open("./dil_lam_$(graph)_$(samp)/AUC$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1]) 
    end
    open("./dil_lam_$(graph)_$(samp)/Ov$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + T + 1]) 
    end
    open("./dil_lam_$(graph)_$(samp)/L1_$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + 2*T + 2]) 
    end
    open("./dil_lam_$(graph)_$(samp)/MSE$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + 3*T + 3]) 
    end
end
open("./dil_lam_$(graph)_$(samp)/Fe.txt","w") do io
    writedlm(io,inf_out[:,:,4*T + 6]) 
end
open("./dil_lam_$(graph)_$(samp)/infλ.txt","w") do io
    writedlm(io,inf_out[:,:,4*T + 7]) 
end
open("./dil_lam_$(graph)_$(samp)/infγ.txt","w") do io
    writedlm(io,inf_out[:,:,4*T + 8]) 
end